In [68]:
import os
import sys

import dotenv
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [69]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from quantnote_query_api.query_api_client import QuantNoteQueryApi

In [70]:
dotenv.load_dotenv(".env")
AUTH_TOKEN = os.getenv("AUTH_TOKEN")
client = QuantNoteQueryApi(auth_token=AUTH_TOKEN, api_version="v1")
client

In [71]:
pair_symbol = 'BTCUSDT'
exchange = 'binance'
pair_query = """SELECT *
    FROM cex.{exchange}.pairs
    WHERE symbol = '{symbol}';""".format(exchange=exchange, symbol=pair_symbol)

pair = client.get_data_response(pair_query)
pair

Waiting for response till limit:   0%|          | 0/600 [00:00<?, ?it/s]

,active,exchange,id,symbol
0,True,binance,1,BTCUSDT


In [72]:
query = """SELECT *
from screener.screener.time_frame_analyses anal
where anal.exchange = '{exchange}'
  and token_id = {pair_id}
  and timeframe = 'H1'
  and updated_at >= timestamp '2023-03-15 08:00:00'
  and update_insert > 0;""".format(exchange=exchange, pair_id=pair['id'][0])

analytics = client.get_data_response(query)
analytics['updated_at'] = pd.to_datetime(analytics['updated_at'])
analytics.set_index('updated_at')
analytics

Waiting for response till limit:   0%|          | 0/600 [00:00<?, ?it/s]

,ad_osc,adx,aroon_down,aroon_up,b_band_lower,b_band_middle,b_band_upper,bearish_engulfing,bullish_engulfing,cci,...,three_white_soldiers,timeframe,token_id,trend,triangle_lower,triangle_type,triangle_upper,update_insert,updated_at,williams_r
0,104.816060,32.632123,24,56,23651.013120,24989.5365,26328.059880,-1,-1,-22.577404,...,-1,H1,1,-0.005293,0,,0,1678867200,2023-03-15 08:00:00+00:00,-57.489459
1,91.335469,32.445404,20,52,23653.112320,24937.8460,26222.579680,-1,-1,-18.062917,...,-1,H1,1,-0.007285,0,,0,1678870800,2023-03-15 09:00:00+00:00,-2.522250
2,-1120.879072,30.914035,16,48,23639.939097,24871.3450,26102.750903,-1,-1,-45.826105,...,-1,H1,1,-0.010155,0,,0,1678874400,2023-03-15 10:00:00+00:00,-38.099184
3,2886.145097,29.641533,12,44,23661.511833,24806.2025,25950.893167,-1,-1,-48.962708,...,-1,H1,1,-0.012706,0,,0,1678878000,2023-03-15 11:00:00+00:00,-40.845833
4,4263.005430,29.542885,8,40,23706.495073,24741.0600,25775.624927,-1,-1,-28.358681,...,-1,H1,1,-0.014573,0,,0,1678881600,2023-03-15 12:00:00+00:00,-40.845833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
738,641.801437,57.783305,80,8,30158.170527,30590.5725,31022.974473,9,42,67.400129,...,-1,H1,1,0.006264,0,,0,1681473640,2023-04-14 12:00:40.003000+00:00,-27.519482
739,-263.190503,55.061327,76,4,30190.747249,30605.3030,31019.858751,10,43,29.200248,...,-1,H1,1,0.006232,0,,0,1681477240,2023-04-14 13:00:40.003000+00:00,-53.205351
740,60.168402,50.884428,72,0,30211.133130,30620.0875,31029.041870,11,44,30.410619,...,-1,H1,1,0.006208,0,,0,1681480840,2023-04-14 14:00:40.012000+00:00,-40.358276
741,-35.723365,46.477140,68,0,30227.241467,30628.0430,31028.844533,12,45,-5.076485,...,-1,H1,1,0.006174,0,,0,1681484440,2023-04-14 15:00:40.006000+00:00,-77.422164


In [73]:
tohlcv_query = """SELECT time, open, high, low, close, volume, trades
FROM cex.{exchange}.candles_hourly
WHERE time > timestamp '2023-03-15 08:00:00'
  and pair_id = {token_id}
ORDER BY time;""".format(exchange=exchange, token_id=pair['id'][0])

tohlcv_resp = client.get_data_response(tohlcv_query)
tohlcv_resp['time'] = pd.to_datetime(tohlcv_resp['time'])
tohlcv_resp.set_index('time')

Waiting for response till limit:   0%|          | 0/600 [00:00<?, ?it/s]

,close,high,low,open,trades,volume
time,,,,,,
2023-03-15 10:00:00+00:00,24491.800000000000000000,24605.000000000000000000,24489.160000000000000000,24553.990000000000000000,88774.000000000000000000,3820.627660000000000000
2023-03-15 11:00:00+00:00,24582.720000000000000000,24628.320000000000000000,24255.000000000000000000,24413.800000000000000000,347059.000000000000000000,18223.848830000000000000
2023-03-15 14:00:00+00:00,24740.180000000000000000,24845.100000000000000000,24717.530000000000000000,24843.170000000000000000,29342.000000000000000000,1735.547420000000000000
2023-03-17 15:00:00+00:00,26421.340000000000000000,26516.460000000000000000,26284.160000000000000000,26339.970000000000000000,417988.000000000000000000,21446.194220000000000000
2023-03-17 16:00:00+00:00,26393.980000000000000000,26573.430000000000000000,26355.410000000000000000,26419.280000000000000000,433014.000000000000000000,21224.266790000000000000
...,...,...,...,...,...,...
2023-04-14 12:00:00+00:00,30616.980000000000000000,30850.000000000000000000,30575.000000000000000000,30801.890000000000000000,80627.000000000000000000,3976.821600000000000000
2023-04-14 13:00:00+00:00,30721.540000000000000000,30749.580000000000000000,30610.030000000000000000,30616.980000000000000000,57262.000000000000000000,3317.948710000000000000
2023-04-14 14:00:00+00:00,30599.890000000000000000,30757.600000000000000000,30483.210000000000000000,30721.540000000000000000,83982.000000000000000000,4968.134890000000000000


In [78]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

candlestick = go.Candlestick(
    x=tohlcv_resp['time'],
    open=tohlcv_resp['open'],
    high=tohlcv_resp['high'],
    low=tohlcv_resp['low'],
    close=tohlcv_resp['close'],
    name="Price"
)

bband_lower = go.Scatter(x=analytics['updated_at'],
                         y=analytics['b_band_lower'],
                         name="bband_lower",
                         mode='lines',
                         line=dict(width=0.5, color='rgb(131, 90, 241)')
                         )
bband_middle = go.Scatter(x=analytics['updated_at'],
                          y=analytics['b_band_middle'],
                          name="bband_middle",
                          mode='lines',
                          line=dict(width=0.5, color='rgb(131, 90, 241)')
                          )
bband_upper = go.Scatter(x=analytics['updated_at'],
                         y=analytics['b_band_upper'],
                         name="bband_upper",
                         mode='lines',
                         line=dict(width=0.5, color='rgb(131, 90, 241)')
                         )

fibonacci23 = go.Scatter(x=analytics['updated_at'],
                         y=analytics['fibonacci23'],
                         name="fibonacci23",
                         mode='lines',
                         line=dict(width=0.8, color='gold')
                         )
fibonacci38 = go.Scatter(x=analytics['updated_at'],
                         y=analytics['fibonacci38'],
                         name="fibonacci38",
                         mode='lines',
                         line=dict(width=0.8, color='gold')
                         )
fibonacci61 = go.Scatter(x=analytics['updated_at'],
                         y=analytics['fibonacci61'],
                         name="fibonacci61",
                         mode='lines',
                         line=dict(width=0.8, color='gold')
                         )
fibonacci78 = go.Scatter(x=analytics['updated_at'],
                         y=analytics['fibonacci78'],
                         name="fibonacci78",
                         mode='lines',
                         line=dict(width=0.8, color='gold')
                         )
fibonacci_low = go.Scatter(x=analytics['updated_at'],
                           y=analytics['fibonacci_low'],
                           name="fibonacci_low",
                           mode='lines',
                           line=dict(width=0.8, color='gold')
                           )
fibonacci_high = go.Scatter(x=analytics['updated_at'],
                            y=analytics['fibonacci_high'],
                            name="fibonacci_high",
                            mode='lines',
                            line=dict(width=0.8, color='gold')
                            )

senkou_span_a = go.Scatter(x=analytics['updated_at'],
                           y=analytics['senkou_span_a'],
                           name="senkou_span_a",
                           mode='lines',
                           line=dict(width=0.5, color='rgb(131, 90, 241)')
                           )

senkou_span_b = go.Scatter(x=analytics['updated_at'],
                           y=analytics['senkou_span_b'],
                           name="senkou_span_b",
                           mode='lines',
                           line=dict(width=0.5, color='gold')
                           )

donchian_lower = go.Scatter(x=analytics['updated_at'],
                            y=analytics['donchian_lower'],
                            name="donchian_lower",
                            mode='lines',
                            line=dict(width=0.5, color='rgb(131, 90, 241)')
                            )

donchian_upper = go.Scatter(x=analytics['updated_at'],
                            y=analytics['donchian_upper'],
                            name="donchian_upper",
                            mode='lines',
                            line=dict(width=0.5, color='gold')
                            )

fig.add_trace(candlestick, secondary_y=True)
# fig.add_trace(bband_lower, secondary_y=True)
# fig.add_trace(bband_middle, secondary_y=True)
# fig.add_trace(bband_upper, secondary_y=True)
# fig.add_trace(fibonacci23, secondary_y=True)
# fig.add_trace(fibonacci38, secondary_y=True)
# fig.add_trace(fibonacci61, secondary_y=True)
# fig.add_trace(fibonacci78, secondary_y=True)
# fig.add_trace(fibonacci_low, secondary_y=True)
# fig.add_trace(fibonacci_high, secondary_y=True)
fig.add_trace(senkou_span_a, secondary_y=True)
fig.add_trace(senkou_span_b, secondary_y=True)
# fig.add_trace(donchian_lower, secondary_y=True)
# fig.add_trace(donchian_upper, secondary_y=True)
# fig.add_trace(tg_sentiment_ewm, secondary_y=False)

fig.update_layout(
    width=1200, height=600,
    title="Analytics {token}".format(token=pair['symbol'][0]),
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    xaxis_rangeslider_visible=False
)

# Set y-axes titles
# fig.update_yaxes(title_text="Sentiment", secondary_y=False)
fig.update_yaxes(title_text="Price ($)", secondary_y=True)

fig.show()

In [75]:
meta_data = analytics.loc[analytics['metadata'].notnull()]
meta_data

,ad_osc,adx,aroon_down,aroon_up,b_band_lower,b_band_middle,b_band_upper,bearish_engulfing,bullish_engulfing,cci,...,three_white_soldiers,timeframe,token_id,trend,triangle_lower,triangle_type,triangle_upper,update_insert,updated_at,williams_r
144,-117.390443,22.087528,8,96,27493.354477,27798.7645,28104.174523,-1,0,-141.251206,...,-1,H1,1,-0.005599,0,,0,1679385600,2023-03-21 08:00:00+00:00,-61.119725
232,-1423.007622,27.089922,0,88,27271.268681,27870.7960,28470.323319,55,29,-91.269663,...,-1,H1,1,-0.000222,0,,0,1679702400,2023-03-25 00:00:00+00:00,-55.598918
236,-247.564374,62.291334,12,72,27220.653397,27738.7390,28256.824603,59,33,-56.190009,...,-1,H1,1,-0.000414,0,,0,1679716800,2023-03-25 04:00:00+00:00,-50.293457
238,-13.040491,59.186666,4,64,27185.607293,27682.4175,28179.227707,61,35,-63.597948,...,-1,H1,1,-0.000515,0,,0,1679724000,2023-03-25 06:00:00+00:00,-45.488738
271,697.910631,25.756748,92,24,27243.717706,27627.1220,28010.526294,1,68,79.194672,...,-1,H1,1,0.003603,0,,0,1679842800,2023-03-26 15:00:00+00:00,-53.330147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
720,324.099669,28.642729,92,12,29824.507741,30152.0310,30479.554259,78,24,117.574178,...,-1,H1,1,0.006669,0,,0,1681408840,2023-04-13 18:00:40.007000+00:00,-27.635728
721,172.010294,30.308211,88,8,29864.120474,30182.0025,30499.884526,79,25,143.192113,...,-1,H1,1,0.006639,0,,0,1681412440,2023-04-13 19:00:40.008000+00:00,-25.196464
727,796.540074,30.812852,100,0,29965.472869,30287.3145,30609.156131,85,31,197.814846,...,-1,H1,1,0.006410,0,,0,1681434040,2023-04-14 01:00:40.015000+00:00,-8.096777
728,1238.403733,27.053688,100,0,29946.552940,30320.8275,30695.102060,86,32,242.292062,...,-1,H1,1,0.006406,0,,0,1681437640,2023-04-14 02:00:40.015000+00:00,-15.593118


In [76]:
import json

for index, row in meta_data.iterrows():
    dictData= json.loads(row['metadata'])
    print(dictData['channel'])
    items = dictData['channel'].split(",")
    items = [item.replace(']', '').replace('[', '') for item in items]
    upper_line = items[0].split()
    lower_line = items[1].split()
    print(upper_line)
    print(lower_line)

[[1.679292e+09 1.6793856e+09 28367.971463097314 27797.475659044292], [1.679292e+09 1.6793856e+09 27607.960676299343 27506.692883403666], "Channel"]
['1.679292e+09', '1.6793856e+09', '28367.971463097314', '27797.475659044292']
['1.679292e+09', '1.6793856e+09', '27607.960676299343', '27506.692883403666']
[[1.6795152e+09 1.6797024e+09 29042.68566096268 27855.264714997746], [1.6795152e+09 1.6797024e+09 27083.45303788644 27345.450289488494], "Channel"]
['1.6795152e+09', '1.6797024e+09', '29042.68566096268', '27855.264714997746']
['1.6795152e+09', '1.6797024e+09', '27083.45303788644', '27345.450289488494']
[[1.6795152e+09 1.6797168e+09 29023.721910953125 27780.614772560883], [1.6795152e+09 1.6797168e+09 27080.798989503222 27370.912020993557], "Channel"]
['1.6795152e+09', '1.6797168e+09', '29023.721910953125', '27780.614772560883']
['1.6795152e+09', '1.6797168e+09', '27080.798989503222', '27370.912020993557']
[[1.6795188e+09 1.679724e+09 28994.828791176715 27743.151966281257], [1.6795188e+09 